<a href="https://colab.research.google.com/github/gremlin97/EVA-8/blob/main/S12/S12_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import glob
import torch
import os

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 2.0.0+cu118 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15101MB, multi_processor_count=40)


In [ ]:
!git clone https://github.com/theschoolofai/YoloV3

Cloning into 'YoloV3'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 159 (delta 0), reused 1 (delta 0), pack-reused 156
Receiving objects: 100% (159/159), 12.75 MiB | 18.15 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
!cp -r /content/drive/MyDrive/Yolo/YoloV3_Weights/weights /content/YoloV3

In [ ]:
# !cp -r /content/drive/MyDrive/Yolo/YoloV3_Weights/data /content/YoloV3
!cp -r /content/drive/MyDrive/Yolo/YoloV3_Weights/data_final/data /content/YoloV3

In [ ]:
%cd YoloV3

/content/YoloV3


In [ ]:
!ls

 annotation_tool   output	  results.txt	     train.py
 cfg		   __pycache__	  test_batch0.png   'ubdivisions=1'
 data		   README.md	  test.py	     utils
 detect.py	   results.json   ting		     vim.exe.stackdump
 models.py	   results.png	  train_batch0.png   weights


Change line 874 in utils.py to : 

```ns = int(np.ceil(bs ** 0.5))  # number of subplots```

Added the following two lines at 472,473 in utils.py :

```
t = t.to(targets.device)
a = a.to(targets.device)
```

*For custom number of classes change the filters and class values as follows:*

* Search for 'filters=255' (you should get entries entries). Change 255 to 27 = (4+1+4)*3
* Search for 'classes=80' and change all three entries to 'classes=4'


*If we are using very few samples do the following changes:*
* burn_in to 100
* max_batches to 5000
* steps to 4000,4500

In [ ]:
!python train.py --data data/customdata/custom.data --batch 10 --cache --cfg cfg/yolov3-custom.cfg --epochs 50 --nosave

Namespace(epochs=50, batch_size=10, accumulate=4, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', multi_scale=False, img_size=[512], rect=False, resume=False, nosave=True, notest=False, evolve=False, bucket='', cache_images=True, weights='weights/yolov3-spp-ultralytics.pt', name='', device='', adam=False, single_cls=False)
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15101MB)

2023-04-05 10:16:51.151670: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 10:16:52.417568: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 

In [ ]:
%cd YoloV3

/content/YoloV3


In [ ]:
#!python train.py --data data/smalcoco/smalcoco.data --batch 10 --cache --epochs 25 --nosave

In [ ]:
# Copy video from which frames need to be extracted into FFMEPG into video folder
%cd ..
%mkdir video
%cp -r /content/drive/MyDrive/Yolo/gojo.mp4 /content/video
%mkdir frames

In [ ]:
# Extract frames using ffmpeg
!ffmpeg -i /content/video/gojo.mp4 /content/frames/%04d.png

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [ ]:
# !python detect.py --conf-thres 0.3 --output out
# Run custom yolo on the video frames and save it to out folder
!python detect.py --conf-thres 0.3 --source /content/frames --output out

Namespace(cfg='cfg/yolov3-custom.cfg', names='data/customdata/custom.names', weights='weights/last.pt', source='/content/frames', output='out', img_size=512, conf_thres=0.3, iou_thres=0.6, fourcc='mp4v', half=False, device='', view_img=False, save_txt=False, classes=None, agnostic_nms=False, augment=False)
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15101MB)

Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
image 1/941 /content/frames/0001.png: 320x512 1 Gojos, Done. (0.039s)
image 2/941 /content/frames/0002.png: 320x512 1 Gojos, Done. (0.030s)
image 3/941 /content/frames/0003.png: 320x512 1 Gojos, Done. (0.030s)
image 4/941 /cont

In [ ]:
%cd ..

/content


In [ ]:
# Use ffmepg to convert back the frames into a video wit24 fps (generally animes have 24 fps)
!ffmpeg -framerate 24 -i /content/YoloV3/out/%04d.png -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p output.mp4

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [ ]:
# Copy output to drive
!cp /content/output.mp4 /content/drive/MyDrive/Yolo